In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer, AdamW, BertForSequenceClassification
import torch


In [ ]:
data_1 = pd.read_csv("/train_data.csv")


In [ ]:
data_2 = pd.read_csv("/train_asamee.csv")

In [ ]:
# Load the dataset
data_1 = pd.read_csv("/train_data.csv")
data_2 = pd.read_csv("/train_asamee.csv")
# Bangla and asamee dataset together
concate_train_data = pd.concat([data_1,data_2])
concate_train_data

,S. No.,text,task_1
0,1,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দি...,NOT
1,2,"প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ, এমপি, এদ...",NOT
2,3,বিশ্ব জুড়ে বুদ্ধিজীবিদের সঙ্গে সম্পর্ক থাকে ব...,HOF
3,4,প্রতি বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতা...,NOT
4,5,সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ করে দেয় ...,HOF
...,...,...,...
4031,4032,তাতে কেইদিনমান mosti কৰি আহক গৈ।,HOF
4032,4033,তাইক এটা গুলী মাৰি সিপুৰীলৈ পঠাই দিয়ক,HOF
4033,4034,এনে গেলা পিক পেলোৱা লোক বহুত আছে,HOF
4034,4035,অসমীয়া ভাষা ৰক্ষা কৰিব গেদাই।,NOT


In [ ]:
# Convert "task_1" labels to 0 and 1
concate_train_data['task_1'] = concate_train_data['task_1'].apply(lambda label: 1 if label == "HOF" else 0)
concate_train_data

,S. No.,text,task_1
0,1,তবে শুনলাম মমতা ব্যানার্জি কোটা পদ্ধতি তুলে দি...,0
1,2,"প্রত্যেক দলের প্রত্যেক মন্ত্রী এমএলএ, এমপি, এদ...",0
2,3,বিশ্ব জুড়ে বুদ্ধিজীবিদের সঙ্গে সম্পর্ক থাকে ব...,1
3,4,প্রতি বছর শত শত মানুষ হত্যা করছে সীমান্তে গণতা...,0
4,5,সমাজের সবথেকে দূষিত মানুষ এরাই খারাপ করে দেয় ...,1
...,...,...,...
4031,4032,তাতে কেইদিনমান mosti কৰি আহক গৈ।,1
4032,4033,তাইক এটা গুলী মাৰি সিপুৰীলৈ পঠাই দিয়ক,1
4033,4034,এনে গেলা পিক পেলোৱা লোক বহুত আছে,1
4034,4035,অসমীয়া ভাষা ৰক্ষা কৰিব গেদাই।,0


In [ ]:
# Split the dataset into train and test
train_data, test_data = train_test_split(data, test_size=0.0001, random_state=42)

# Set device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load Bangla BERT model and tokenizer

model_name = "saroarj/BanglaHateBert"
# model_name = "sagorsarker/bangla-bert-base"
bert_model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize and encode the texts
train_texts = train_data['text'].tolist()
train_labels = train_data['task_1'].tolist()

In [ ]:
tokenized_train_texts = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
input_ids = tokenized_train_texts["input_ids"].to(device)
attention_mask = tokenized_train_texts["attention_mask"].to(device)
labels = torch.tensor(train_labels).to(device)

# Define BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

# Optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 10
total_steps = len(input_ids) * num_epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# Data loader
batch_size = 2
train_dataset = TensorDataset(input_ids, attention_mask, labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids_batch, attention_mask_batch, labels_batch = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids_batch, attention_mask=attention_mask_batch, labels=labels_batch)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Adjust learning rate
    scheduler.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss}")

# Save the trained model (skipping for brevity)

# Evaluation on test data
test_texts = test_data['text'].tolist()
test_labels = test_data['task_1'].tolist()
tokenized_test_texts = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
input_ids_test = tokenized_test_texts["input_ids"].to(device)
attention_mask_test = tokenized_test_texts["attention_mask"].to(device)
labels_test = torch.tensor(test_labels).to(device)

model.eval()
with torch.no_grad():
    outputs = model(input_ids_test, attention_mask=attention_mask_test)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()

accuracy = accuracy_score(test_labels, predicted_labels)
conf_matrix = confusion_matrix(test_labels, predicted_labels)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10 - Average Loss: 0.6844700487175336
Epoch 2/10 - Average Loss: 0.5148956756327405
Epoch 3/10 - Average Loss: 0.23883776368812606
Epoch 4/10 - Average Loss: 0.0669825642615049
Epoch 5/10 - Average Loss: 0.046355409160545
Epoch 6/10 - Average Loss: 0.010615648417443582
Epoch 7/10 - Average Loss: 0.03094652943001266
Epoch 8/10 - Average Loss: 0.03603439736070868
Epoch 9/10 - Average Loss: 0.004537384514151199
Epoch 10/10 - Average Loss: 0.0006033148898091491
Accuracy: 0.0
Confusion Matrix:
 [[0 1]
 [0 0]]


In [ ]:
# Load the new data
new_data = pd.read_csv("test_data.csv")  # Replace with the path to your new data CSV file

# Tokenize and encode the new text data
new_texts = new_data['text'].tolist()
tokenized_new_texts = tokenizer(new_texts, padding=True, truncation=True, return_tensors="pt")
input_ids_new = tokenized_new_texts["input_ids"].to(device)
attention_mask_new = tokenized_new_texts["attention_mask"].to(device)

# Testing/Prediction
model.eval()

with torch.no_grad():
    outputs = model(input_ids_new, attention_mask=attention_mask_new)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1).tolist()

# Convert predicted labels to human-readable form
label_map = {0: "NOT", 1: "HOF"}

for text, label in zip(new_texts, predicted_labels):
    print(f"Text: {text}")
    print(f"Predicted Label: {label_map[label]}")
    print("=" * 40)

Text: তাতে তোর কী? বেশি চুলকুনি নারে সালা ?
Predicted Label: NOT
Text: অবশ্য চটি চাটা হতে হবে
Predicted Label: HOF
Text: KKR এর প্রসঙ্গ তুললে শাহরুখের ছবি দেয়া কি বাধ্যতামূলক? শাহরুখ KKR এর সহ-মালিক হতে পারে, ক্রিকেটার নয়। ক্রিকেট এবং ক্রিকেটারকে ভালবেসে আপামর জনতা খেলা দেখে ,দলের মালিককে দেখে নয়।
Predicted Label: NOT
Text: হ্যা হ্যা ভিখিরি বলতেই পাশের দেশের লোকের লাগছে। যার যা নাম ধরে ডাকলেই চিনতে পারে।
Predicted Label: NOT
Text: এটা নিজের মেয়ের নাম রাখবেন আর এই নাম ধরে ডাকবেন .....😁😅😆
Predicted Label: NOT
Text: পিগি চপস মানে বাংলায় ওইটাই বোঝায় 😂😂😂😂😂 আমার মেয়ে হলে নাম রাখব "লকেট" 😂😂😂😂😂
Predicted Label: HOF
Text: মা-ছেলে দুজনকেই ভালো লাগছে।
Predicted Label: HOF
Text: ইয়ে মাথায় ওঠেনি। এগুলো সবই পরিকল্পনা করে বিদ্বেষ ছাড়ানো
Predicted Label: NOT
Text: হারামি কেন বলেন বলুন তো?
Predicted Label: NOT
Text: খবর প্রচার করুন ভাল কথা তবে খেয়াল রাখবেন যেসব মানুষ অশ্লীল ভাষা ব্যবহার করে তাদেরকে আমন্ত্রণ করবেন না
Predicted Label: NOT
Text: এখবরের কোন সত্যতা নেই, মনগড়া খবর পরিবেশন করা এসব ফালতু 

In [ ]:
# new_data

In [ ]:
# predicted_labels

In [ ]:
import pandas as pd


# Convert predicted labels to human-readable form
label_map = {0: "NOT", 1: "HOF"}
predicted_labels_human = [label_map[label] for label in predicted_labels]

result_df = pd.DataFrame({"S. No.": range(1, len(predicted_labels) + 1), "task1": predicted_labels_human})

# Save the DataFrame as a CSV file
result_df.to_csv("predicted_label_10.csv", index=False)